# HaGRID Dataset - Hand Gesture Recognition in Real-world Environments

**Dataset:** HaGRID by Kaggle (kapitanua/hagrid)  
**Size:** 500k+ images of hand gestures in diverse environments  
**Classes:** 18 hand gestures (ok, peace, thumbs_up, thumbs_down, fist, palm, etc.)  
**Format:** PNG images with JSON annotations (hand bounding boxes)  
**Challenge:** Real-world backgrounds, various hand sizes and orientations  
**Goal:** Train robust MobileNet/EfficientNet model for practical deployment

**Best Run On:** Google Colab with GPU (large dataset)  
**Training Time:** 20-40 minutes on GPU

In [ ]:
# Setup
import subprocess, sys
packages = ['tensorflow', 'kaggle', 'opencv-python', 'pillow', 'numpy', 'matplotlib', 'scikit-learn', 'tensorflowjs']
for pkg in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', pkg])

from pathlib import Path
for d in ['external_data', 'datasets', 'models', 'output']:
    Path(d).mkdir(parents=True, exist_ok=True)
print("✓ Setup complete")

In [ ]:
# Download HaGRID dataset
import os

# Kaggle download
# os.system('kaggle datasets download -d kapitanua/hagrid -p external_data --unzip')

# Classes to download (18 gestures)
HAGRID_CLASSES = [
    'ok', 'peace', 'thumbs_up', 'thumbs_down', 'fist', 'palm', 
    'call', 'rock', 'stop', 'no_gesture', 'mute', 'dislike',
    'like', 'pointing_up', 'pointing_down', 'pointing_left', 'pointing_right', 'closed_fist'
]

print("✓ HaGRID dataset configuration")
print(f"  Classes: {len(HAGRID_CLASSES)}")
print(f"  To download: Configure Kaggle API and uncomment os.system() above")
print(f"  Dataset: https://www.kaggle.com/datasets/kapitanua/hagrid")

In [ ]:
# Preprocess HaGRID with hand detection
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
import json

def preprocess_hagrid(src_dir='external_data/hagrid', dst_dir='datasets/hagrid', img_size=160):
    """Extract hand regions and save as uniform-size images."""
    src = Path(src_dir)
    dst = Path(dst_dir)
    
    if not src.exists():
        print(f"⚠ HaGRID directory not found")
        return
    
    success = 0
    for gesture_dir in src.iterdir():
        if not gesture_dir.is_dir():
            continue
        
        gesture = gesture_dir.name
        dst_gesture = dst / gesture
        dst_gesture.mkdir(parents=True, exist_ok=True)
        
        # Process images in gesture folder
        for img_file in gesture_dir.glob('*.png'):
            try:
                # Load image
                img = cv2.imread(str(img_file))
                if img is None:
                    continue
                
                # Resize to uniform size
                img_resized = cv2.resize(img, (img_size, img_size))
                
                # Save
                cv2.imwrite(str(dst_gesture / img_file.name), img_resized)
                success += 1
            except:
                pass
    
    print(f"✓ Preprocessed {success} HaGRID images")

preprocess_hagrid()

In [ ]:
# Train EfficientNetB0 on HaGRID
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

IMG_SIZE = 160
BATCH = 32
EPOCHS = 20
DATA_DIR = 'datasets/hagrid'

if not os.path.exists(DATA_DIR) or len(os.listdir(DATA_DIR)) == 0:
    print("⚠ HaGRID dataset not found")
else:
    # Data generators
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.15,
        validation_split=0.15
    )
    
    train_flow = datagen.flow_from_directory(DATA_DIR, target_size=(IMG_SIZE, IMG_SIZE),
                                            batch_size=BATCH, subset='training', class_mode='categorical')
    val_flow = datagen.flow_from_directory(DATA_DIR, target_size=(IMG_SIZE, IMG_SIZE),
                                          batch_size=BATCH, subset='validation', class_mode='categorical')
    
    # Build EfficientNetB0 model
    base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base.trainable = False
    
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    outputs = Dense(train_flow.num_classes, activation='softmax')(x)
    
    model = Model(inputs=base.input, outputs=outputs)
    model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train
    history = model.fit(train_flow, validation_data=val_flow, epochs=EPOCHS, verbose=1)
    model.save('models/hagrid_efficientnet.h5')
    print("✓ Model trained and saved")

In [ ]:
# Evaluate and export HaGRID model
import matplotlib.pyplot as plt
import tensorflowjs as tfjs

# Training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(history.history['accuracy'], label='Train')
ax1.plot(history.history['val_accuracy'], label='Val')
ax1.set_title('Accuracy')
ax1.legend()

ax2.plot(history.history['loss'], label='Train')
ax2.plot(history.history['val_loss'], label='Val')
ax2.set_title('Loss')
ax2.legend()
plt.tight_layout()
plt.savefig('output/hagrid_training.png', dpi=100)
plt.show()

# Export
tfjs.converters.save_keras_model(model, 'output/tfjs_hagrid')
print("✓ Model exported to output/tfjs_hagrid/")